# Inference-time self-supervised learning demo

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from inference_ssl import *
from icon_eval import eval_preds


## Show results on Pascal VOC FG/BG segmentation

In [2]:
# imports
from util.icon_dataset import Transformation, Resize
from icon_eval import ToTensorUnscaled

from util.voc_dataset import VOCSegmentationSubFgBg, HBBoxTransform

Prepare the data and model:

In [3]:
args = get_inference_args(notebook=True)
args.batch_size = 4
root = os.path.join(dataset_dir, 'voc')
    # add some space around the objects if possible
bbox_trans = HBBoxTransform(range=(0.3, 0.3))
# sub = 'all'
im_transform = tfms.Compose([Transformation(train=False, size=args.input_size)])
mask_transform = tfms.Compose([Resize(args.input_size, args.input_size)])
sub = 'cat&dog'
dataset = VOCSegmentationSubFgBg(root=root,
                                 sub=sub,
                                 transform=im_transform,
                                 target_transform=mask_transform,
                                 bbox_transform=bbox_trans)

print(args)
batch_size = 1 if args.tta_iter_num > 0 else args.batch_size

model = load_model(args)


BBOX GT /Users/panda/Technion/datasets/voc/VOCdevkit/VOC2012/ImageSets/Segmentation/bboxes_cat&dog.npy does not exist, it will be created now.
BBOX GT has been created successfully!
Namespace(model_name='unet3_bce_iou', model_pick='best', model='mae_vit_base_patch16_seg_conv_unet', unet_depth=3, data_path='/Users/panda/Technion/datasets', batch_size=4, input_size=384, num_workers=4, dataset_name='SOD', ssl_iter_num=15)
<All keys matched successfully>


In [4]:
gauss_noise_fun = lambda x: (x + torch.randn_like(x) * 0.2)
poisson_noise_fun = lambda x: (x + torch.poisson(x * 0.9) / 0.9).clamp(0, 1)
gauss_fun = lambda x: torch.einsum('chw -> hwc', F.gaussian_blur(torch.einsum('hwc -> chw', x), kernel_size=5, sigma=3))
contrast_fun = lambda x: torch.einsum('chw -> hwc', F.adjust_contrast(torch.einsum('hwc -> chw', x), 2))
brightness_fun = lambda x: torch.einsum('chw -> hwc', F.adjust_brightness(torch.einsum('hwc -> chw', x), 2))

Run optimization:

In [ ]:
idxs = [76, 77, 80, 100]
idxs = np.arange(120, 125)

for idx in idxs:
    img, gt, clses, name = dataset[idx]

    gt = (gt > 0).astype(int)
    # reload model
    model = load_model(args)
    model.eval() # makes sure layers like batchnorm or dropout are in eval mode 
    model.freeze_decoder()
    # iterative ssl optimization
    
    (preds_seg, preds_rec) = optimize_im(img=gauss_noise_fun(torch.tensor(img).to(device)), model=model, num_it=20, thresh=0.1,
                                         lr=1e-2, bs=args.batch_size, mask_ratio=0.5, debug=True)
    maes_seg, maes_rec, ious_seg = eval_preds(preds_seg, preds_rec, gt, img)
    plot_results(name, img, gt, preds_seg, preds_rec, maes_seg, maes_rec)

<All keys matched successfully>
iter 0: mse_rec = 0.11135008931159973, mse_init_rec = 0.11135008931159973
iter 1: mse_rec = 0.1079242080450058, mse_init_rec = 0.1079242080450058
iter 2: mse_rec = 0.11442174017429352, mse_init_rec = 0.11442174017429352
iter 3: mse_rec = 0.10951656103134155, mse_init_rec = 0.10951656103134155
iter 4: mse_rec = 0.10846338421106339, mse_init_rec = 0.10846338421106339
iter 5: mse_rec = 0.12167128920555115, mse_init_rec = 0.12167128920555115
iter 6: mse_rec = 0.12181101739406586, mse_init_rec = 0.12181101739406586
iter 7: mse_rec = 0.11962278932332993, mse_init_rec = 0.11962278932332993
iter 8: mse_rec = 0.12482544034719467, mse_init_rec = 0.12482544034719467
iter 9: mse_rec = 0.12003853917121887, mse_init_rec = 0.12003853917121887
iter 10: mse_rec = 0.11341214179992676, mse_init_rec = 0.11341214179992676
iter 11: mse_rec = 0.11557735502719879, mse_init_rec = 0.11557735502719879


In [5]:
pred

NameError: name 'pred' is not defined